In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
from tqdm import tqdm
import seaborn as sns
%matplotlib inline
import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2

from keras import backend as K

from keras.models import load_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()
# data_type is 'train', 'test' and 'bad_train'
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)
batch_size = 32
# not feature (depth) vec
tgs_generator = Generator(train_ids = train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size, feature_out=False)

# steps_per_epoch
steps_per_epoch = len(train_ids) //  batch_size
validation_steps = len(val_ids) //  batch_size
# validation batch will be all validation

# validation_steps = 1
print(steps_per_epoch)
print(validation_steps)

97
24


In [53]:
images, depths, masks = tgs_generator.get_val_data()

In [30]:
im_id = train_ids[0]

In [31]:
im = cv2.imread('./data/train/images/' + im_id + '.png')

In [36]:
from skimage.transform import resize
im_resize = resize(im, (128, 128), mode='constant', preserve_range=True)

In [37]:
im_resize_return = resize(im_resize, (101, 101), mode='constant', preserve_range=True)

In [42]:
train_ids

array(['ad1e8ef844', '33f32ff406', '085e7a048e', ..., 'de08c5a3b7',
       'a50ac7d1e2', '9e7d89d4c0'], dtype=object)

In [43]:
X_train, X_feat_train, Y_train, Y_no_sult, Y_salt_cls = train_util.get_data(data_type='train', size=(128, 128, 3))

100%|██████████| 3887/3887 [00:49<00:00, 78.01it/s]


In [44]:
from sklearn.cross_validation import train_test_split
x_train, x_test, x_feat_train, x_feat_test, y_train, y_test, y_no_sult_train, y_no_sult_test = train_test_split(X_train, X_feat_train, Y_train, Y_no_sult, test_size=0.2, random_state=42, stratify= Y_salt_cls)

In [45]:
x_train

array([[[[0.52156866, 0.52156866, 0.52156866],
         [0.49803922, 0.49803922, 0.49803922],
         [0.49411765, 0.49411765, 0.49411765],
         ...,
         [0.49019608, 0.49019608, 0.49019608],
         [0.49411765, 0.49411765, 0.49411765],
         [0.50980395, 0.50980395, 0.50980395]],

        [[0.5411765 , 0.5411765 , 0.5411765 ],
         [0.5176471 , 0.5176471 , 0.5176471 ],
         [0.50980395, 0.50980395, 0.50980395],
         ...,
         [0.49019608, 0.49019608, 0.49019608],
         [0.49803922, 0.49803922, 0.49803922],
         [0.50980395, 0.50980395, 0.50980395]],

        [[0.5411765 , 0.5411765 , 0.5411765 ],
         [0.52156866, 0.52156866, 0.52156866],
         [0.5137255 , 0.5137255 , 0.5137255 ],
         ...,
         [0.49411765, 0.49411765, 0.49411765],
         [0.50980395, 0.50980395, 0.50980395],
         [0.52156866, 0.52156866, 0.52156866]],

        ...,

        [[0.5254902 , 0.5254902 , 0.5254902 ],
         [0.5137255 , 0.5137255 , 0.5137255 ]

In [22]:
X_train, X_feat_train, Y_train, Y_no_salt, Y_salt_cls = train_util.get_data(data_type='train', size=(128, 128, 3))

 12%|█▏        | 464/3887 [00:05<00:41, 81.68it/s]


KeyboardInterrupt: 

In [ ]:
inter2 = np.histogram2d(Y_train[0].flatten(), Y_train[0].flatten(), bins=(2, 2))

In [ ]:
Y_train[:10].flatten().shape

In [ ]:
a = np.array([[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0]])
b = np.array([[0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1]])

In [ ]:
inter = np.histogram2d(a.flatten(), b.flatten(), bins=(2, 2))[0]

In [ ]:
area_true = np.histogram(a, bins=2)[0]
area_pred = np.histogram(b, bins=2)[0]

In [ ]:
area_true = np.expand_dims(area_true, -1)
area_pred = np.expand_dims(area_pred, 0)

In [ ]:
area_true

In [ ]:
inter = inter[1:, 1:]

In [ ]:
area_pred

In [ ]:
area_true + area_pred - inter

In [ ]:
union = area_true + area_pred -inter

In [ ]:
uni = union[1:, 1:]

In [ ]:
uni[uni == 0]

In [ ]:
iou = inter / uni

In [ ]:
matches = iou > 0.5

In [ ]:
np.sum(matches, axis=0) == 0

In [ ]:
def iou_coef(y_true, y_pred, smooth=1):
    """
    IoU = (|X & Y|)/ (|X or Y|)
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return intersection
#     union = K.sum((y_true,-1) + K.sum(y_pred,-1)) - intersection
#     return (intersection + smooth) / ( union + smooth)

In [ ]:
a = np.array([[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0]])
b = np.array([[0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 1, 1], [0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 1, 1]])

In [ ]:
# Immutable
変更不可　int float string tuple
mutable
list numpy array dict
# byval byref

In [ ]:
def testFunc(x, y):
    print(id(x))
    print(id(y))
    x.append('hello')
    
    return c, d

In [ ]:
a = [1, 2, 3, 4]

In [ ]:
print(a)

In [ ]:
f, g = testFunc(a, a)

In [131]:
# print(a, f, g)
# a [1, 2, 3, 4]
# f [1, 2, 3, 4, 'hello']
# g [1, 2, 3, 4, 'world']

In [141]:
print(a)

[1, 2, 3, 4, 'hello', 'world']


In [150]:
def test_func2(ab):
    ab += 3
    print(id(ab))
    
    return ab

In [151]:
g = 0

In [152]:
id(g)

140523687799296

In [153]:
cd = test_func2(g)

140523687799392


In [184]:
x_train = np.array([0, 1, 2, 3, 4, 5], dtype='float')

In [185]:
x_val = np.array([4, 1, 2, 8, 4, 6], dtype='float')

In [188]:
def norm(x_train, x_val):
    mean = x_train.mean()
    std = x_train.std()
    
    x_train -= mean
    x_train /= std
    
    x_val -= mean
    x_val /= std
    
    return x_train, x_val

In [189]:
test = norm(x_train, x_val)

In [193]:
x_test = norm(x_train, x_train)

In [194]:
x_train

array([-1.46385011, -0.87831007, -0.29277002,  0.29277002,  0.87831007,
        1.46385011])

In [143]:
 print(g)

0


In [195]:
import copy

In [197]:
copy_x = copy.deepcopy(x_train)

In [198]:
id(x_train)

140520807478576

In [199]:
id(copy_x)

140520807430144